## Métodos Numéricos II: Ejercicios Práctica 2

#### Autor: Juan Manuel Rodríguez Gómez

In [1]:
import numpy as np

import sympy as sp

import matplotlib.pyplot as plt
%matplotlib inline

from tabulate import tabulate # Si no se encuentra el módulo, hacer "conda install tabulate" en la consola con el actual
                              # environment activado (se puede abrir desde Anaconda)

## Ejercicio 1
#### Obtenga mediante interpolación en el espacio $\mathbb{P}_2$ una fórmula para aproximar $f''(a)$ del tipo combinación de $f(a-h)$, $f(a)$ y $f(a+h)$.

In [2]:
f = sp.Function('f')
a,h = sp.symbols('a,h')

x = [a-h,a,a+h]  # Nodos de interpolación
y = [f(x[0]),f(x[1]),f(x[2])]  # Valores interpolados
z = sp.Symbol('z')  # Utilizaremos z como variable simbólica

y

[f(a - h), f(a), f(a + h)]

In [3]:
p0 = y[0]  # Vamos a ir construyendo el polinomio de interpolación
p1 = p0 + (z-x[0])/(x[1]-x[0])*(y[1]-y[0]) # usando la idea de Newton

In [4]:
D = sp.Symbol('D')   # D será la correspondiente diferencia dividida
p2 = p1 + (z-x[0])*(z-x[1])*D # D = f[x0,x1,x2]
p2  # Ya tenemos la expresión genérica de dicho polinomio de grado 2

D*(-a + z)*(-a + h + z) + f(a - h) + (f(a) - f(a - h))*(-a + h + z)/h

In [5]:
p2.subs({z:x[0]})==y[0],p2.subs({z:x[1]})==y[1] 
# Comprobamos que p2 interpola a y0 e y1

(True, True)

In [6]:
# Pero tenemos aún que garantizar que p(x2) = y2

sol2 = sp.solve(p2.subs({z:x[2]})-y[2],D)  # Para ello resolvemos
sol2    # la ecuación correspondiente y despejamos el valor de D

[(-2*f(a) + f(a - h) + f(a + h))/(2*h**2)]

In [7]:
D = sol2[0]

p2 = p1 + (z-x[0])*(z-x[1])*D
p2 # Polinomio final de interpolación 

f(a - h) + (f(a) - f(a - h))*(-a + h + z)/h + (-a + z)*(-a + h + z)*(-2*f(a) + f(a - h) + f(a + h))/(2*h**2)

In [8]:
p2.subs({z:x[2]})==y[2] # Comprobamos que p2 interpola a y2

True

In [9]:
# Obtenemos la primera derivada

sp.diff(p2,z).subs({z:a}).simplify() # Derivo y evalúo en z=a

(-f(a - h) + f(a + h))/(2*h)

In [10]:
# Obtenemos la segunda derivada

sp.diff(p2,z,2) 
# Nótese que en este caso, ni siquiera hemos tenido
# que evaluar dicha derivada en el punto x=a, al ser esta constante.

(-2*f(a) + f(a - h) + f(a + h))/h**2

## Ejercicio 2
#### Con la fórmula obtenida en el ejercicio 1, halle una tabla de aproximaciones y errores de $f_1''(2.5)$, siendo $f_1(x)=x^x$, para $h=10^{-i},\; i=1,\ldots,5.$

In [11]:
def f1(z):
    """función x**x"""
    return (z**z)

In [12]:
f1(z)

z**z

In [13]:
# Valor exacto de f1'' en el punto a = 2.5

a = 2.5 
valor_exacto_derivadasegundaf1 = (sp.diff(f1(z),z,2)).subs({z:a}) # La segunda derivada de f1 es x**x(ln(x)+1)**2 + x**(x-1)
valor_exacto_derivadasegundaf1

40.2416648563875

In [14]:
# Mediante la fórmula numérica obtenida en el ejercicio 1
# calculamos las aproximaciones de f1'' en a = 2.5
# con h desde 10^1 hasta el valor 10^(-n).

n = 5 
valores_aproximados_derivadasegundaf1 = np.array([(-2*f1(a)+f1(a-10**(-j))+f1(a+10**(-j)))/(10**(-j))**2 for j in range(1,n+1)])
print(valores_aproximados_derivadasegundaf1)

[40.4205683  40.24345023 40.24168271 40.24166476 40.24164113]


In [15]:
# Errores cometidos (es decir, valores aproximados menos los valores exactos)

errores = abs(np.array(valores_aproximados_derivadasegundaf1) - valor_exacto_derivadasegundaf1)
print(errores)  

[0.178903441570853 0.00178537455153815 1.78514991731049e-5
 1.00363045874019e-7 2.37259090170028e-5]


In [16]:
# Tabla con los errores cometidos

print('     Exacto       Aproximacion        Error')
for x in range(0,n):
    print ('{0:2f} {3} {1:3f} {3} {2:4f} '.format(valor_exacto_derivadasegundaf1, valores_aproximados_derivadasegundaf1[x], errores[x], '|'))

     Exacto       Aproximacion        Error
40.2416648563875 | 40.420568 | 0.178903441570853 
40.2416648563875 | 40.243450 | 0.00178537455153815 
40.2416648563875 | 40.241683 | 0.0000178514991731049 
40.2416648563875 | 40.241665 | 0.000000100363045874019 
40.2416648563875 | 40.241641 | 0.0000237259090170028 


## Ejercicio 3
#### Sea $f_2(x)=\frac{x^2+40}{x+\sqrt{5x}+7}$. Calcule una tabla que recoja las derivadas de $f_2$ en $x_i=1,2,\ldots,10$, utilizando alguna de las fórmulas de derivación numérica de primer orden obtenidas al inicio de la práctica, con $h=10^{-3}$, y muestre al mismo tiempo el error cometido en cada punto. Repita el ejercicio con la fórmula centrada obtenida para la derivada primera y, finalmente, para la obtenida en el ejercicio 1 (con respecto a la segunda derivada).

In [17]:
def f2(z):
    """función (x**2+40)/(x+(5*x)**(1/2)+7)"""
    return (z**2+40)/(z+(5*z)**(1/2)+7)

In [18]:
f2(z)

(z**2 + 40)/(2.23606797749979*z**0.5 + z + 7)

In [19]:
# Valores exactos de f2' en los puntos x_i = 1,2,...,10

n = 10 
valores_exactos_derivadaprimeraf2 = np.array([(sp.diff(f2(z),z)).subs({z:i}) for i in range(1,n+1)])
print(valores_exactos_derivadaprimeraf2) 

[-0.633413841504903 -0.203729991363422 0.0135536765957583
 0.152356382446352 0.250865051903114 0.325234486346073 0.383753089267232
 0.431201820656649 0.470566739057635 0.503824070415537]


In [20]:
# Mediante la fórmula numérica ( f(a+h) - f(a) ) / h
# calculamos las aproximaciones de f2' en x_i = 1,2,...,10
# con h = 10^-3.

valores_aproximados1_derivadaprimeraf2 = np.array([(f2(i+10**(-3))-f2(i))/(10**(-3)) for i in range(1,n+1)])
print(valores_aproximados1_derivadaprimeraf2) 


[-0.63307585 -0.20358841  0.01363783  0.15241383  0.25090736  0.3252672
  0.38377927  0.43122332  0.47058476  0.50383942]


In [21]:
# Errores cometidos (es decir, valores aproximados menos los valores exactos)

errores1 = abs(np.array(valores_aproximados1_derivadaprimeraf2) - np.array(valores_exactos_derivadaprimeraf2))
print(errores1)                       

[0.000337990681841038 0.000141580338231473 8.41579481308807e-5
 5.74471912467844e-5 4.23072889731979e-5 3.27156186125110e-5
 2.61842658260680e-5 2.15041391883886e-5 1.80199924058044e-5
 1.53477178297390e-5]


In [22]:
# Tabla con los errores cometidos

print('       Exacto       Aproximacion        Error')
for x in range(0,n):
    print ('{0:2f} {3} {1:3f} {3} {2:4f} '.format(valores_exactos_derivadaprimeraf2[x], valores_aproximados1_derivadaprimeraf2[x], errores1[x], '|'))

       Exacto       Aproximacion        Error
-0.633413841504903 | -0.633076 | 0.000337990681841038 
-0.203729991363422 | -0.203588 | 0.000141580338231473 
0.0135536765957583 | 0.013638 | 0.0000841579481308807 
0.152356382446352 | 0.152414 | 0.0000574471912467844 
0.250865051903114 | 0.250907 | 0.0000423072889731979 
0.325234486346073 | 0.325267 | 0.0000327156186125110 
0.383753089267232 | 0.383779 | 0.0000261842658260680 
0.431201820656649 | 0.431223 | 0.0000215041391883886 
0.470566739057635 | 0.470585 | 0.0000180199924058044 
0.503824070415537 | 0.503839 | 0.0000153477178297390 


In [23]:
# Mediante la fórmula centrada para la derivada primera ( f(a+h) - f(a-h) ) / 2h
# calculamos las aproximaciones de f2'' en x_i = 1,2,...,10
# con h = 10^-3.

valores_aproximados2_derivadaprimeraf2 = np.array([(f2(i+10**(-3))-f2(i-10**(-3)))/(2*10**(-3)) for i in range(1,n+1)])
print(valores_aproximados2_derivadaprimeraf2)

[-0.63341398 -0.20373002  0.01355366  0.15235638  0.25086505  0.32523448
  0.38375309  0.43120182  0.47056674  0.50382407]


In [24]:
# Errores cometidos (es decir, valores aproximados menos los valores exactos)

errores2 = abs(np.array(valores_aproximados2_derivadaprimeraf2) - np.array(valores_exactos_derivadaprimeraf2))
print(errores2)

[1.41948942822268e-7 2.98522312980332e-8 1.22133764213217e-8
 6.46658449010573e-9 3.92622495626327e-9 2.59752386266854e-9
 1.82080633903681e-9 1.33376959476905e-9 1.01006225605715e-9
 7.85527420887888e-10]


In [25]:
# Tabla con los errores cometidos

print('       Exacto       Aproximacion        Error')
for x in range(0,n):
    print ('{0:2f} {3} {1:3f} {3} {2:4f} '.format(valores_exactos_derivadaprimeraf2[x], valores_aproximados2_derivadaprimeraf2[x], errores2[x], '|'))

       Exacto       Aproximacion        Error
-0.633413841504903 | -0.633414 | 0.000000141948942822268 
-0.203729991363422 | -0.203730 | 0.0000000298522312980332 
0.0135536765957583 | 0.013554 | 0.0000000122133764213217 
0.152356382446352 | 0.152356 | 0.00000000646658449010573 
0.250865051903114 | 0.250865 | 0.00000000392622495626327 
0.325234486346073 | 0.325234 | 0.00000000259752386266854 
0.383753089267232 | 0.383753 | 0.00000000182080633903681 
0.431201820656649 | 0.431202 | 0.00000000133376959476905 
0.470566739057635 | 0.470567 | 0.00000000101006225605715 
0.503824070415537 | 0.503824 | 0.000000000785527420887888 


In [26]:
# Valores exactos de f2'' en los puntos x_i = 1,2,...,10

valores_exactos_derivadasegundaf2 = np.array([(sp.diff(f2(z),z,2)).subs({z:i}) for i in range(1,n+1)])
print(valores_exactos_derivadasegundaf2)

[0.676265098285376 0.283220364176106 0.168340319928121 0.114907312895053
 0.0846224302869937 0.0654364313639429 0.0523721743690358
 0.0430109449028751 0.0360420057237485 0.0306970066620211]


In [27]:
# Mediante la fórmula numérica obtenida en el ejercicio 1 ( f(a+h) + f(a-h) - 2f(a) ) / h^2
# calculamos las aproximaciones de f2'' en x_i = 1,2,...,10
# con h = 10^-3.

valores_aproximados_derivadasegundaf2 = np.array([(f2(i+10**(-3))+f2(i-10**(-3))-2*f2(i))/(10**(-3))**2 for i in range(1,n+1)])
print(valores_aproximados_derivadasegundaf2)

[0.67626526 0.28322038 0.16834032 0.11490732 0.08462243 0.06543643
 0.05237217 0.04301095 0.036042   0.03069701]


In [28]:
# Errores cometidos (es decir, valores aproximados menos los valores exactos)

errores3 = abs(np.array(valores_aproximados_derivadasegundaf2) - np.array(valores_exactos_derivadasegundaf2))
print(errores3)

[1.62394166891566e-7 1.63053464619090e-8 3.08648318014804e-9
 2.76749589911418e-9 5.53403864400615e-10 1.79698290969643e-9
 2.16043211098693e-10 1.01309156202989e-9 7.87627567333526e-10
 8.35879736954537e-10]


In [29]:
# Tabla con los errores cometidos

print('     Exacto       Aproximacion        Error')
for x in range(0,n):
    print ('{0:2f} {3} {1:3f} {3} {2:4f} '.format(valores_exactos_derivadasegundaf2[x], valores_aproximados_derivadasegundaf2[x], errores3[x], '|'))

     Exacto       Aproximacion        Error
0.676265098285376 | 0.676265 | 0.000000162394166891566 
0.283220364176106 | 0.283220 | 0.0000000163053464619090 
0.168340319928121 | 0.168340 | 0.00000000308648318014804 
0.114907312895053 | 0.114907 | 0.00000000276749589911418 
0.0846224302869937 | 0.084622 | 0.000000000553403864400615 
0.0654364313639429 | 0.065436 | 0.00000000179698290969643 
0.0523721743690358 | 0.052372 | 0.000000000216043211098693 
0.0430109449028751 | 0.043011 | 0.00000000101309156202989 
0.0360420057237485 | 0.036042 | 0.000000000787627567333526 
0.0306970066620211 | 0.030697 | 0.000000000835879736954537 


## Ejercicio 4
#### Divida el intervalo $[1,2]$ en 100 partes iguales y aplique las fórmulas del rectángulo, Simpson y trapecio compuestas para aproximar la integral en dicho intervalo de $f_1$. Compare dichos resultados.

In [30]:
def formrectangizda(f,a,b,nx):
    """fórmula compuesta de los rectangulos a izquierda"""
    h = (b-a)/nx
    return h*sum([f(a+i*h) for i in range(0,nx)])

In [31]:
a = 1
b = 2
n = 100

In [32]:
f1_rect_izq = formrectangizda(f1,a,b,n)
f1_rect_izq

2.0354943390855573

In [33]:
def formrectangdcha(f,a,b,nx):
    """fórmula compuesta de los rectangulos a derecha"""
    h = (b-a)/nx
    return h*sum([f(a+(i+1)*h) for i in range(0,nx)])

In [34]:
f1_rect_der = formrectangdcha(f1,a,b,n)
f1_rect_der

2.065494339085557

In [35]:
def formrectangptomedio(f,a,b,nx):
    """fórmula compuesta de los rectangulos de punto medio"""
    h = (b-a)/nx
    return h*sum([f(a+(i+1/2)*h) for i in range(0,nx)])

In [36]:
f1_rect_pto_medio = formrectangptomedio(f1,a,b,n)
f1_rect_pto_medio

2.050422182392515

In [37]:
def formtrapecios(f,a,b,nx):
    """fórmula compuesta de los trapecios"""
    h = (b-a)/nx
    return h/2*(f(a)+2*sum([f(a+i*h) for i in range(1,nx)])+f(b))

In [38]:
f1_trapecios = formtrapecios(f1,a,b,n)
f1_trapecios

2.0504943390855574

In [39]:
def Simpsoncompuesta(f,a,b,m):
    """fórmula de Simpson compuesta"""
    h= (b-a)/(2*m)
    P = sum([f(a+2*i*h) for i in range(1,m)])
    I = sum([f(a+(2*i-1)*h) for i in range(1,m+1)])
    E = f(a)+f(b)
    return h/3*(E+2*P+4*I)

In [40]:
f1_simpson = Simpsoncompuesta(f1,a,b,n)
f1_simpson

2.0504462346235295

In [41]:
from scipy.integrate import quad # Usamos quad porque sympy no es capaz de calcular el valor exacto
                                 # de la integral usando sp.integrate

valor_exacto_f1, abserr_f1 = quad(f1, a, b)
valor_exacto_f1, abserr_f1

(2.050446234534731, 2.2764526203364124e-14)

In [42]:
resultados_integracion_f1 = [['Rectangulos a izquierda', f1_rect_izq, abs(f1_rect_izq-valor_exacto_f1)],
         ['Rectangulos a derecha', f1_rect_der, abs(f1_rect_der-valor_exacto_f1)],
         ['Rectangulos de punto medio', f1_rect_pto_medio, abs(f1_rect_pto_medio-valor_exacto_f1)],
         ['Trapecios', f1_trapecios, abs(f1_trapecios-valor_exacto_f1)],
         ['Simpson', f1_simpson, abs(f1_simpson-valor_exacto_f1)]]

print("Valor \"exacto\": {}\n".format(valor_exacto_f1))

print("Fórmula                 Aproximación   Error")
print(tabulate(resultados_integracion_f1))

Valor "exacto": 2.050446234534731

Fórmula                 Aproximación   Error
--------------------------  -------  -----------
Rectangulos a izquierda     2.03549  0.0149519
Rectangulos a derecha       2.06549  0.0150481
Rectangulos de punto medio  2.05042  2.40521e-05
Trapecios                   2.05049  4.81046e-05
Simpson                     2.05045  8.87983e-11
--------------------------  -------  -----------


Vemos que el resultado dado por la fórmula de Simpson es la que más se aproxima al valor exacto.

## Ejercicio 5
#### Repita el ejercicio 4 para $f_2$. 

In [43]:
f2_rect_izq = formrectangizda(f2,a,b,n)
f2_rect_izq

3.778523202782093

In [44]:
f2_rect_der = formrectangdcha(f2,a,b,n)
f2_rect_der

3.774646194132547

In [45]:
f2_rect_pto_medio = formrectangptomedio(f2,a,b,n)
f2_rect_pto_medio

3.7765793274267083

In [46]:
f2_trapecios = formtrapecios(f2,a,b,n)
f2_trapecios

3.77658469845732

In [47]:
f2_simpson = Simpsoncompuesta(f2,a,b,n)
f2_simpson

3.7765811177702457

In [48]:
valor_exacto_f2, abserr_f2 = quad(f2, a, b)
valor_exacto_f2, abserr_f2

(3.77658111776791, 4.192847311310543e-14)

In [49]:
resultados_integracion_f2 = [['Rectangulos a izquierda', f2_rect_izq, abs(f2_rect_izq-valor_exacto_f2)],
         ['Rectangulos a derecha', f2_rect_der, abs(f2_rect_der-valor_exacto_f2)],
         ['Rectangulos de punto medio', f2_rect_pto_medio, abs(f2_rect_pto_medio-valor_exacto_f2)],
         ['Trapecios', f2_trapecios, abs(f2_trapecios-valor_exacto_f2)],
         ['Simpson', f2_simpson, abs(f2_simpson-valor_exacto_f2)]]

print("Valor \"exacto\": {}\n".format(valor_exacto_f2))

print("Fórmula                 Aproximación   Error")
print(tabulate(resultados_integracion_f2))

Valor "exacto": 3.77658111776791

Fórmula                 Aproximación   Error
--------------------------  -------  -----------
Rectangulos a izquierda     3.77852  0.00194209
Rectangulos a derecha       3.77465  0.00193492
Rectangulos de punto medio  3.77658  1.79034e-06
Trapecios                   3.77658  3.58069e-06
Simpson                     3.77658  2.33591e-12
--------------------------  -------  -----------


Al igual que el ejercicio anterior, vemos que el resultado dado por la fórmula de Simpson es la que más se aproxima al valor exacto.

## Ejercicio 6
#### Sea $f_3(x)=x^{15} e^x$ en $[0,2]$. Vamos a dividir el intervalo en $10\times 2^n$ subintervalos, es decir, $10,\,20,\,40,\, 80,\ldots $ y a aplicar la fórmula de Simpson compuesta hasta que la diferencia entre dos aproximaciones consecutivas (por ejemplo, podrían ser con $20$ y $40$ subintervalos) sea menor que $10^{-2}$, dando en tal caso por buena la última aproximación obtenida. Programe y calcule dicha aproximación. Compare ambas aproximaciones con el valor exacto.

In [50]:
def f3(z):
    """función x**(15)*sp.exp(x)"""
    return z**(15)*sp.exp(z)

In [51]:
f3(z)

z**15*exp(z)

In [52]:
a = 0
b = 2;
tolerancia = 10**(-2)

In [53]:
def subintervalos(n):
    return 10*2**n

def SimpsonConTolerancia(f, a, b, tol):
    simpson0 = Simpsoncompuesta(f,a,b,subintervalos(1))
    simpson1 = Simpsoncompuesta(f,a,b,subintervalos(2))
    n = 2
    print("n =", n, "   error =", float(simpson0-simpson1))
    while (abs(simpson0-simpson1)>= tol):
        n = n+1
        simpson0 = simpson1
        simpson1 = Simpsoncompuesta(f,a,b,subintervalos(n))
        print("n =", n, "   error =", float(simpson0-simpson1))
        
    return simpson1

In [54]:
valor_aproximado_integralf3 = SimpsonConTolerancia(f3, a, b, tolerancia)

n = 2    error = 4.054081166325991
n = 3    error = 0.2562843870996927
n = 4    error = 0.0160635850614006
n = 5    error = 0.001004691558945894


In [55]:
float(valor_aproximado_integralf3)

27062.702480891214

In [56]:
valor_exacto_integralf3 = sp.integrate(f3(z),[z,a,b])
float(valor_exacto_integralf3)

27062.702413899602

In [57]:
error = abs(valor_aproximado_integralf3 - valor_exacto_integralf3)
float(error)

5.627667466096787e-05

Vemos que el error cometido es muy pequeño, luego, la aproximación es buena.

## Ejercicio 7
#### Calcule las fórmulas gaussianas con $2$ y $3$ nodos,en el intervalo $[-1,1]$, siendo la función peso el valor absoluto de la variable. Aplíquelas para aproximar la función $x\; e^x$ en $[-1,1]$ y compare los resultados con el valor exacto (organizando los cálculos de forma adecuada).

In [58]:
x = sp.Symbol('x')

def w(x):
    """función peso"""
    return abs(x)

w(-1), w(0), w(1)

(1, 0, 1)

In [59]:
def f(x):
    return x*sp.exp(x)

In [60]:
a = -1
b = 1;

In [61]:
def formgaussiana(n, f, w, a, b):
    grexact = 2*n-1 # Grado exactitud
    
    p = sp.symbols('p0:'+ str(n)) # Tupla de n nodos
    nodos = list(p)
    
    c = sp.symbols('c0:'+ str(n)) # Tupla de n coeficientes
    coefs = list(c)
    
    incogs = coefs + nodos
    
    ecs = [np.dot([(z**i).subs({z:nodos[j]}) for j in range(n)],coefs)-sp.integrate(w(x)*x**i,(x,a,b)) for i in range(grexact+1)]

    solsGauss = sp.solve(ecs,incogs)
    
    for i in range(n):
        coefs[i] = solsGauss[0][i]
        nodos[i] = solsGauss[0][n+i]
        
    formGauss = np.dot([f(nodos[i]) for i in range(n)],coefs)
    
    return formGauss

In [62]:
# Fórmula gaussiana con 2 nodos

gauss_dos_nodos = formgaussiana(2, f, w, -1, 1)
gauss_dos_nodos

-sqrt(2)*exp(-sqrt(2)/2)/4 + sqrt(2)*exp(sqrt(2)/2)/4

In [63]:
# Fórmula gaussiana con 3 nodos

gauss_tres_nodos = formgaussiana(3, f, w, -1, 1)
gauss_tres_nodos

-sqrt(6)*exp(-sqrt(6)/3)/8 + sqrt(6)*exp(sqrt(6)/3)/8

In [64]:
gauss_dos_nodos = float(gauss_dos_nodos)
gauss_tres_nodos =float(gauss_tres_nodos)
valor_exacto = sp.integrate(w(x)*f(x), [x, a, b])

In [65]:
table = [[2, gauss_dos_nodos, valor_exacto, abs(gauss_dos_nodos-valor_exacto)],
         [3, gauss_tres_nodos, valor_exacto, abs(gauss_tres_nodos-valor_exacto)]]

print(tabulate(table, headers=["n","Gaussiana", "Exacto", "Error"]))

  n    Gaussiana    Exacto        Error
---  -----------  --------  -----------
  2     0.542721  0.557679  0.0149582
  3     0.557437  0.557679  0.000241959


## Ejercicio 8
#### Programar las técnicas de integración de Romberg y adaptativa, para después aplicarlas a la aproximación de la siguiente integral $$\int_a^b p(x)\, dx$$ siendo  $\;a=\displaystyle\min_{0\leq i\leq 7}{d_i}$, $\;b=\displaystyle\max_{0\leq i\leq 7}{d_i}$ y  $$p(x)=d_0 + d_1 x + d_2 x^2 + d_3 x^3+ d_4 x^4 + d_5 x^5 + d_6 x^6 + d_7 x^7 $$ (siendo $d_0, d_1, \ldots, d_7$ los dígitos de su DNI, pasaporte o tarjeta de residente).

In [66]:
def p(x):
    """polinomio 4+9*x+5*x**2+5*x**3+9*x**4+4*x**5+9*x**6+4*x**7"""
    return 4+9*x+5*x**2+5*x**3+9*x**4+4*x**5+9*x**6+4*x**7

In [67]:
a = 4
b = 9

In [68]:
def Romberg(f, a, b, tol):
    N = 1; n = 1; h = b-a
    R = np.zeros((50, 50))
    R[1][1] = h/2*(f(a)+f(b))
    
    while( abs(R[N-1][N-1]-R[N][N]) > tol):
        x = np.linspace(a+h/2, b-h/2, n)
        R[N+1][1] = (R[N][1]+h*sum(f(x)))/2
        N = N+1; n = 2*n; h = h/2;
        
        for k in range (2,N+1):
            coef = 4*(k-1)
            R[N][k] = (coef*R[N][k-1] - R[N-1][k-1])/(coef-1)
        #print(R[N][N])
    return R[N][N]  

In [69]:
romberg = Romberg(p, a, b, 10**(-2))
romberg

28084369.345255036

In [70]:
def Adaptativa(f, a, b, epsilon):
    def S(a,b):
        m = (a+b)/2
        h = (b-a)/2
        return h/3 * (f(a)+4*f(m)+f(b))

    m = (a+b)/2
    simpson = S(a,m)+S(m,b)
    if( abs(S(a,b)-simpson) < 10*epsilon): 
        return simpson
    else: 
        return Adaptativa(f,a,m,epsilon/2)+ Adaptativa(f,m,b,epsilon/2)

In [71]:
adaptativa = Adaptativa(p,a,b,10**(-2))
adaptativa

28084369.34781106

In [72]:
valor_exacto = float(sp.integrate(p(x), [x, a, b]))
valor_exacto

28084369.345238097

In [73]:
table=[[valor_exacto, romberg, abs(romberg-valor_exacto), adaptativa, abs(adaptativa-valor_exacto)]]
print(tabulate(table, headers=["Exacto", "Romberg", "Error_Romberg", "Adaptativa", "Error_Adaptativa"]))

     Exacto      Romberg    Error_Romberg    Adaptativa    Error_Adaptativa
-----------  -----------  ---------------  ------------  ------------------
2.80844e+07  2.80844e+07      1.69389e-05   2.80844e+07          0.00257296


Fijándonos en el error, vemos que la integración de Romberg ha obtenido mejores resultados que la integración Adaptativa ya que el error cometido al aplicar la integración de Romberg es mucho menor que el error al aplicar la integración Adaptativa.